In [1]:
## There are 4 sizes that are used as type parameters for the needed protocols.
## using the `Dim` class, we can ensure that our objects align in their dimensionalities.

## C: The number of conditional variables

from modugant.matrix.dim import Dim

conditions = Dim[0](0)
## L: The number of latent variables
latent = Dim[10](10)
## G: The number of generated features
generated = Dim[5](5)
## D: The number of real features to discriminate
dim = Dim[5](5)
## The batch size
batch = Dim[8](8)

In [2]:
## A generator maps `(C, L) -> (G)`

## Instatiate a sequential generator

from modugant.generators import SequentialGenerator
from modugant.matrix import Matrix
from modugant.matrix.ops import zeros

conditions = Dim[0](0)
latent = Dim[10](10)
generated = Dim[5](5)
batch = Dim[8](8)

generator = SequentialGenerator(
    conditions,
    latent,
    generated,
    steps = [10, 10, 5],
    learning = 0.01,
    gamma = 0.1,
    step = 100
)

## The generator can be used to sample from the latent space
## Pass in a 0-width condition of the desired length
## Use `Matrix.zeros` for type/dimension alignment
print(generator.sample(zeros((batch, conditions))))

tensor([[0.4984, 0.4999, 0.4512, 0.3986, 0.5031],
        [0.4985, 0.4938, 0.4426, 0.3993, 0.5048],
        [0.4982, 0.4945, 0.4427, 0.3994, 0.5049],
        [0.5047, 0.4912, 0.4405, 0.4068, 0.4965],
        [0.4983, 0.4972, 0.4468, 0.3991, 0.5040],
        [0.4982, 0.4945, 0.4427, 0.3994, 0.5049],
        [0.4989, 0.4972, 0.4479, 0.3991, 0.5033],
        [0.5055, 0.4893, 0.4374, 0.4085, 0.4957]], grad_fn=<SigmoidBackward0>)


In [3]:
## Instantiate a residual generator

from modugant.generators import ResidualGenerator

conditions = Dim[0](0)
latent = Dim[10](10)
generated = Dim[5](5)
batch = Dim[8](8)

generator = ResidualGenerator(
    conditions,
    latent,
    generated,
    steps = [10, 10, 5],
    learning = 0.01,
    decay = 0.001
)

print(generator.sample(zeros((batch, conditions))))

tensor([[-0.6909,  0.2056,  0.8441, -0.9408,  0.2882],
        [ 0.6643,  0.3398, -0.3775,  0.4350, -0.4729],
        [-0.3590, -0.0150,  0.4493, -0.5246, -0.2024],
        [ 0.3525,  0.3371,  0.1595, -0.0891, -0.3798],
        [ 0.4280,  0.2691,  0.1975, -0.4721, -0.1675],
        [ 0.5172, -0.0845, -0.4411,  0.6337, -0.3657],
        [-0.2107,  0.7293,  0.1717, -0.3085,  0.1956],
        [ 0.1047,  0.3106,  0.4389, -0.5143, -0.1937]],
       grad_fn=<TanhBackward0>)


In [ ]:
## Create a custom generator that fully implements the `Generator` protocol

from typing import Self, cast, override

from torch import Tensor, no_grad
from torch.nn import Linear, Module
from torch.optim.adam import Adam

from modugant import Generator
from modugant.device import Device
from modugant.matrix.dim import One
from modugant.matrix.ops import cat, normal


class CustomGenerator[L: int, G: int](Module, Generator[One, L, G]):
    def __init__(self, latent: L, generated: G):
        super().__init__()
        self._conditions = Dim.one()
        self._latents = latent
        self._intermediates = generated
        self._model = Linear(latent + 1, generated)
        self._optimizer = Adam(self.parameters(), lr = 0.01)
    @override
    def forward(self, x: Tensor) -> Tensor:
        return self._model(x)
    @override
    def sample[N: int](self, condition: Matrix[N, One]) -> Matrix[N, G]:
        ## Make sure not to set device, it will be controlled by context!!
        noise = normal(0, 1, (condition.shape[0], self._latents))
        inputs = cat((condition, noise), dim = 1, shape = (condition.shape[0], self._latents + 1))
        outputs = self.forward(inputs)
        return Matrix.load(outputs, shape = (condition.shape[0], self._intermediates))
    @override
    def update(self, loss: Tensor) -> None:
        self.zero_grad()
        _ = loss.backward()
        cast(None, self._optimizer.step())
    @override
    def reset(self) -> None:
        with no_grad():
            for module in self.modules():
                if isinstance(module, Linear):
                    module.reset_parameters()
    @override
    def restart(self) -> None:
        self._optimizer = Adam(self.parameters(), lr = 0.01)
    @override
    def move(self, device: Device) -> Self:
        return self.to(device)
    @override
    def train(self, mode: bool = True) -> Self:
        return self.train(mode)
    @property
    @override
    def rate(self) -> float:
        return self._optimizer.param_groups[0]['lr']

latent = Dim[10](10)
generated = Dim[5](5)
batch = Dim[8](8)

generator = CustomGenerator(latent, generated)

print(generator.sample(normal(0, 1, (batch, Dim.one()))))

tensor([[ 0.4077,  0.2677,  0.3967,  0.0790,  0.2311],
        [ 0.3939,  0.3259,  0.1252,  0.0035, -0.3723],
        [ 0.2634, -1.0318, -0.7844,  0.7270, -0.3338],
        [ 0.3618,  0.5002,  0.9131,  0.2635,  0.0556],
        [ 0.0288,  1.2959, -0.6457, -0.0047,  0.2984],
        [ 0.3421,  0.2940, -0.5743,  0.3495, -0.7437],
        [-0.9578,  0.5221, -0.1214,  1.5171,  0.9110],
        [ 0.3485,  0.2320,  0.4842,  0.4727, -0.1676]],
       grad_fn=<AddmmBackward0>)


In [5]:
## Create a custom generator by extending `BasicGenerator` abstract class

from modugant.generators import BasicGenerator


class CustomGenerator[L: int, G: int](BasicGenerator[One, L, G]):
    def __init__(self, latent: L, generated: G):
        super().__init__(Dim.one(), latent, generated)
        self._model = Linear(latent + 1, generated)
        self._optimizer = Adam(self.parameters(), lr = 0.01)
    @override
    def _latent[N: int](self, batch: N) -> Matrix[N, L]:
        return normal(0, 1, (batch, self._latents))
    @override
    def restart(self) -> None:
        self._optimizer = Adam(self.parameters(), lr = 0.01)

latent = Dim[10](10)
generated = Dim[5](5)
batch = Dim[8](8)

generator = CustomGenerator(latent, generated)

print(generator.sample(normal(0, 1, (batch, Dim.one()))))

tensor([[-1.2854,  1.1247,  1.7791, -1.0458,  0.0507],
        [-0.2544,  0.7251,  1.6548, -0.0276, -0.7635],
        [ 0.0733,  0.0490, -0.0177, -0.8428, -0.1150],
        [-0.1081,  0.0540, -0.7972, -0.2182,  0.5202],
        [ 0.2779, -0.4938,  0.0600, -0.5387, -0.0948],
        [-0.3534,  0.7919,  0.8774, -0.9429, -0.6382],
        [-0.5599,  1.0466,  0.9090, -1.0613, -0.0298],
        [ 0.2569,  0.5898,  0.1190, -0.4355,  0.3592]],
       grad_fn=<AddmmBackward0>)


In [6]:
## Add a scheduler to the generator

from torch.optim.lr_scheduler import StepLR


class ScheduledGenerator[L: int, G: int](CustomGenerator[L, G]):
    def __init__(self, latent: L, generated: G):
        super().__init__(latent, generated)
        self._scheduler = StepLR(self._optimizer, step_size = 100, gamma = 0.1)
    @override
    def update(self, loss: Matrix[One, One]) -> None:
        super().update(loss)
        self._scheduler.step()
    @override
    def restart(self) -> None:
        super().restart()
        self._scheduler = StepLR(self._optimizer, step_size = 100, gamma = 0.1)